# Q-Learning Algorithm

_Demonstrating Q-Learning algorithm with suitable assumptions for a problem statement._

This is based on standard _Hello Word_ Reinforcement Learnining program for a maze problem written by Dr. James McCaffrey, Senior Research Software Engineer at Mirosoft Research. Required changes were made to make this more relevant to the need.

![](simple_maze_problem.png)

The 3x5 maze has 15 cells, numbered from 0 to 14. The goal is to get from cell 0 in the upper left corner, to cell 14 in the lower right corner, in the fewest moves. Only left, right, up, or down movement is allowed, but not diagonal one.
    
The demo program sets up a representation of the maze in memory and then uses a Q-learning algorithm to find a _Q_ matrix. Higher the value, better it is. The row indices are the "from" cells and the column indices are the "to" cells. If the starting cell is 0, then scanning that row shows the largest Q value is 0.02 at to-cell 5. Then at cell 5, the largest value in the row is 0.05 at to-cell 10. The process continues until the program reaches the goal state at cell 14.

In [1]:
# imports required packages

import pandas as pd
import numpy as np

## Utility Functions

In [2]:
def get_possible_next_states(state, F, states_count):
    """
    For a given cell state 'state', the function uses the feasibility matrix F to
    determine which states can be reached, and returns those states as a list. For example, 
    if state = 5, the return list is [0, 6, 10] for given maze.
    
    Parameters
    ----------
    state: int
        (Zero-based) index for representing state for which all possible states are required
        
    F: ndarray
        Feasibility matrix
        
    states_count: int
        Total number of states in the maze
        
    Returns
    -------
    Out: list
        List of all possible states
        
    """
    
    possible_next_states = []
    for i in range(states_count):
        if F[state, i] == 1: 
            possible_next_states.append(i)
    
    return possible_next_states

In [3]:
def get_random_next_state(state, F, states_count):
    """
    For a given state 'state', all possible next states are determined, and then 
    one of those states is randomly selected. For example, if state = 5, then 
    candidates are [0, 6, 10] for given maze. The call to np.randomint() 
    returns a random value from 0 to 2 considering index of random next state.

    Parameters
    ----------
    state: int
        (Zero-based) index for representing state for which all possible states are required
        
    F: ndarray
        Feasibility matrix
        
    states_count: int
        Total number of states in the maze
    
    """
    
    possible_next_states = get_possible_next_states(state, F, states_count)
    next_state = possible_next_states[np.random.randint(0, len(possible_next_states))]
    
    return next_state

## Initialization

In [4]:
# Loads up feasibility matrix denoted as F
F = np.loadtxt("./feasibility_matrix.csv", dtype="int", delimiter=',')

# Prints the matrix
print(F)

[[0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [5]:
# Loads up reward matrix denoted as R
R = np.loadtxt("./reward_matrix.csv", dtype="float", delimiter=',')

# Prints the matrix; Note that reward is provided only when
# goal is reached. Refer location (9, 14).
print(R)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [6]:
# Initializes quality matrix, denoted by Q, with all zeros

Q = np.zeros(shape=[15,15], dtype=np.float32)

# Prints the Q matrix
display(pd.DataFrame(Q, dtype=float).style.format(precision=2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Training

In [7]:
def train(F, R, Q, gamma, lr, goal_state, states_count, episodes):
    """
    Computes the Q-matrix using Bellman equation, which is an extension of 
    standard Q function, that considers trade-off between exploitation and exploration.
    
    Parameters
    -----------
    F: Feasibility matrix
    R: Reward matrix
    Q: Q matrix
    gamma: Discount factor
    lr: learning rate through which trade-off between exploitation and exploration is expressed.
    goal_state: The goal state the agent should reach
    states_count: Total number of states in the maze
    episodes: Total number of episodes during traning
    
    """
    
    for i in range(0, episodes):
        # Selects a random start state
        current_state = np.random.randint(0, states_count)

        # Continues till goal state is reached
        while(True):
            # Selects a random next state from the current state
            next_state = get_random_next_state(current_state, F, states_count)
            
            # Gets all possible states from that next state
            possible_next_next_states = get_possible_next_states(next_state, F, states_count)

            # Compares the Q value between two possible next states
            max_Q = -9999.99
            for j in range(len(possible_next_next_states)):
                next_next_state = possible_next_next_states[j]
                q = Q[next_state, next_next_state]
                if q > max_Q:
                    max_Q = q
            
            # Updates the Q value using Bellman equation [refer maze image caption]
            Q[current_state][next_state] = \
                ((1 - lr) * Q[current_state][next_state]) + (lr * (R[current_state][next_state] + (gamma * max_Q)))

            # Changes state by considering next state as current state and 
            # the training continues till goal state is reached
            current_state = next_state
            
            if current_state == goal_state:
                break

In [8]:
# Sets hyperparameters

gamma = 0.5        # discount factor
lr = 0.5           # learning_rate
goal_state = 14
states_count = 15
episodes = 1000

In [9]:
np.random.seed(42)

# starts training
train(F, R, Q, gamma, lr, goal_state, states_count, episodes)

In [10]:
# Prints Q matrix generated out of training
display(pd.DataFrame(Q, dtype=float).style.format(precision=2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.62,0.00,2.50,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00
5,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.16,0.00,0.00
8,0.00,0.00,0.00,1.25,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,2.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00


In [11]:
# The function walk() 

def print_shortest_path(start_state, goal_state, Q):
    """
    Finds shortest path in printable form
    
    Parameters
    ----------
    start_state: int
        Index of the starting state
        
    goal_state: int
        Index of the goal state
        
    Q: ndarray
        Learned Q matrix
    
    """
    
    current_state = start_state
    print(str(current_state) + "->", end="")
    
    # Loops till goal is reached and keeps on tracing the path
    while current_state != goal_state:
        # Chooses the best state from possible states and keep on printing
        next_state = np.argmax(Q[current_state])
        print(str(next_state) + "->", end="")
        
        # Considers next state as current state and continiues till goal is reached
        current_state = next_state
    
    print("Goal Reached.\n")

## Testing

In [12]:
# Performs few tests for agent to get the shortest path

start_state = 8
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 13
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 6
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

start_state = 1
print("Best path to reach goal from state {0} to goal state {1}".format(start_state, goal_state))
print_shortest_path(start_state, goal_state, Q)

Best path to reach goal from state 8 to goal state 14
8->3->4->9->14->Goal Reached.

Best path to reach goal from state 13 to goal state 14
13->12->7->8->3->4->9->14->Goal Reached.

Best path to reach goal from state 6 to goal state 14
6->5->10->11->12->7->8->3->4->9->14->Goal Reached.

Best path to reach goal from state 1 to goal state 14
1->0->5->10->11->12->7->8->3->4->9->14->Goal Reached.

